# Wat this note can do? 
* This not can make CSV file that summarize inferred selection coefficients
* Selection coefficients shown here are substracted from the TF's coefficients. 
* This process should help to 

Flow of process: 

- reading selection coefficients and csv file that contains HXB2 index. 
- reading sequnce data 
- normalizing selection coefficients. 
- summarizing them in a single csv file. 
- Next, then integrate covariance matrix and numerator across multiple different rhesus macques. 




In [2]:
#Pkg.add("Pkg"); import Pkg; 
ENV["LD_LIBRARY_PATH"] = ""
using Pkg
using Distributed
using DelimitedFiles
using Distances
using StatsBase 
using Profile    
using Random
using Statistics
using LinearAlgebra
using Plots
using Printf
using Measures
using CSV
#using FastaIO
using LaTeXStrings;
using DataFrames
using KernelDensity
rng = Random.MersenneTwister(1234);
PRO = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"];
AA_set = ["-", "A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y", "*"];
AA2NUM = Dict(AA_set[i] => i for i in 1:length(AA_set));
q = length(AA_set);

In [3]:
include("../src/my_HIV_data_cleanign.jl")
include("../src/non_synonymous_N-glycan.jl") 
include("../src/process_HIV_mutation_for_CSV.jl") ;

syn_or_nonsyn_simple (generic function with 1 method)

## CH505 

In [101]:
fname_Human_RM = ["703010505", "RM5695", "RM6072", "RM6701", "RM6699", "RM6697", "RM6070", "RM6703"];

In [102]:
# --- Suppose the CH505's sequence has the complete set of the muations --- #
csv_poly_CH505 = DataFrame(CSV.File( "../mpl/aa_csv/CH505/index-703010505.csv" ));;
seq_TF_aa = csv_poly_CH505.TF; haxb2_TF = csv_poly_CH505.HXB2; seq_consensus_aa = csv_poly_CH505.consensus;
for i_HRM in 1:length(fname_Human_RM)
    # Read CSV
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH505/index-%s.csv", fname_Human_RM[i_HRM])))
    # Read Sequence
    seq_poly = readdlm(@sprintf("../mpl/aa_seq/CH505/%s-poly.num", fname_Human_RM[i_HRM]))
    # Read CSV file 
    coefficients_MPL = readdlm("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH505/"*fname_Human_RM[i_HRM]*"-poly-AA-MPL.dat")
    coefficients_SL = readdlm("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH505/"*fname_Human_RM[i_HRM]*"-poly-AA-SL.dat");

    # Formatting sequencies and get frequency 
    seq_TF = copy(csv_poly.TF[csv_poly.polymorphic .!= "NA"]);
    seq_ensemble = Int.(seq_poly[:, 3:end] .+ 1)
    time_set = Int.(seq_poly[:, 1]);

    seq_TF_num = [AA2NUM[x] for x in seq_TF];
    L_poly = length(seq_TF_num);
    time_unique = sort(unique(time_set))
    n_time_unique = length(time_unique)
    x1 = get_x1_AA(time_set, L_poly, q, seq_ensemble);

    # --- Normalization --- #
    (s_normed_MPL, s_normed_SL) = normalize_coefficient_AA(coefficients_MPL, coefficients_SL, q, L_poly, seq_TF, seq_ensemble, seq_TF_num);

    # --- Formatting CSV file --- #
    idx_sort = sortperm(vec(s_normed_MPL), rev=true)
    i_poly_out_set = []; aa_out_set = []; 
    s_mpl_out_set = []; s_sl_out_set = [];
    hxb2_out_set = []; tf_out_set = []; cons_out_set = []
    mut_naive_out_set = []; # compare with TF
    reversion_out_set = []
    #mut_out_set = []; # consider the previous temporal state
    date_first_detected = [];
    freq_set = [[] for _ in 1:n_time_unique]
    for n in 1:length(idx_sort)
        i_raw = idx_sort[n]
        s_mpl = s_normed_MPL[i_raw]
        s_sl = s_normed_MPL[i_raw]
        i_poly = Int(ceil(i_raw / q))

        a_poly_num = i_raw - (i_poly-1) * q
        a_poly = AA_set[a_poly_num]

        idx_csv = (csv_poly.polymorphic .== string(i_poly))
        i_poly_out = csv_poly.polymorphic[idx_csv][1]
        hxb2_out = csv_poly.HXB2[idx_csv][1]

        if( !( abs(s_mpl)<1e-7) )
            tf_out = csv_poly.TF[idx_csv][1]
            consensus_out = csv_poly.consensus[idx_csv][1]
            date_detected = get_first_detected(time_unique, x1, i_poly, a_poly_num)
            #@show i_poly, a_poly, i_poly_out, hxb2_out, tf_out, consensus_out, date_detected
            if((a_poly==consensus_out) && (a_poly!=tf_out)) push!(reversion_out_set, true) else push!(reversion_out_set, false) end
            push!(hxb2_out_set, hxb2_out)
            push!(i_poly_out_set, i_poly)
            push!(aa_out_set, a_poly)
            push!(tf_out_set, tf_out)
            push!(cons_out_set, consensus_out)
            push!(mut_naive_out_set, @sprintf("%s%s%s", tf_out, hxb2_out, a_poly))
            push!(date_first_detected, date_detected)
            push!(s_mpl_out_set, @sprintf("%.7f", s_mpl))
            push!(s_sl_out_set, @sprintf("%.7f", s_sl))
        
            for i_t in 1:n_time_unique
                push!(freq_set[i_t], @sprintf("%.3f", x1[i_t, i_poly, a_poly_num]))
            end
        
        end
    end
   
    (glycan_plus_set, glycan_minus_set) = get_glycan_plus_minus_AA_seq(seq_TF_aa, haxb2_TF, hxb2_out_set, aa_out_set)
    @printf("N+:%d, N-:%d, N±:%d\n", count(glycan_plus_set), count(glycan_minus_set), count(glycan_plus_set .* glycan_minus_set))
    (V1_set_temp, V2_set_temp, V3_set_temp, V4_set_temp, V5_set_temp, LD_set_temp, CD4BS_set_temp) = get_variable_site_true_false(hxb2_out_set)
    
    df = DataFrame(
        HXB2=hxb2_out_set, 
        polymorphic=i_poly_out_set, 
        PRO=aa_out_set, 
        TF=tf_out_set,
        consensus=cons_out_set,
        reversion=reversion_out_set,
        mutation=mut_naive_out_set, 
        date=date_first_detected,
        V1=V1_set_temp,
        V2=V2_set_temp,
        V3=V3_set_temp,
        V4=V4_set_temp,
        V5=V5_set_temp,
        LD=LD_set_temp,
        CD4BS=CD4BS_set_temp,
        glycan_plus=glycan_plus_set, 
        glycan_minus=glycan_minus_set, 
        s_MPL=s_mpl_out_set, 
        s_SL=s_sl_out_set, 
    )
    for i_t in 1:n_time_unique 
        df[!, @sprintf("f_at_%s", time_unique[i_t])] = freq_set[i_t]
    end


    CSV.write(@sprintf("../mpl/aa_csv/CH505/%s-poly.csv", fname_Human_RM[i_HRM]), df);

end

N+:12, N-:97, N±:4
N+:10, N-:47, N±:4
N+:7, N-:63, N±:4
N+:1, N-:15, N±:1
N+:2, N-:16, N±:1
N+:5, N-:43, N±:2
N+:3, N-:38, N±:1
N+:3, N-:21, N±:2


## CH848 

In [103]:
fname_Human_RM = ["703010848", "RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720"];

In [104]:
csv_poly_CH848 = DataFrame(CSV.File( "../mpl/aa_csv/CH848/index-703010848.csv" ));;
seq_TF_aa = csv_poly_CH848.TF; haxb2_TF = csv_poly_CH848.HXB2; seq_consensus_aa = csv_poly_CH848.consensus;

for i_HRM in 1:length(fname_Human_RM)
    # Read CSV
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH848/index-%s.csv", fname_Human_RM[i_HRM])))
    # Read Sequence
    seq_poly = readdlm(@sprintf("../mpl/aa_out/CH848/%s-poly.num", fname_Human_RM[i_HRM]))
    # Read CSV file 
    coefficients_MPL = readdlm("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH848/"*fname_Human_RM[i_HRM]*"-poly-AA-MPL.dat")
    coefficients_SL = readdlm("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH848/"*fname_Human_RM[i_HRM]*"-poly-AA-SL.dat");

    # Formatting sequencies and get frequency 
    seq_TF = copy(csv_poly.TF[csv_poly.polymorphic .!= "NA"]);
    seq_ensemble = Int.(seq_poly[:, 3:end] .+ 1)
    time_set = Int.(seq_poly[:, 1]);

    seq_TF_num = [AA2NUM[x] for x in seq_TF];
    L_poly = length(seq_TF_num);
    time_unique = sort(unique(time_set))
    n_time_unique = length(time_unique)
    x1 = get_x1_AA(time_set, L_poly, q, seq_ensemble);

    # --- Normalization --- #
    (s_normed_MPL, s_normed_SL) = normalize_coefficient_AA(coefficients_MPL, coefficients_SL, q, L_poly, seq_TF, seq_ensemble, seq_TF_num);

    # --- Formatting CSV file --- #
    idx_sort = sortperm(vec(s_normed_MPL), rev=true)
    i_poly_out_set = []; aa_out_set = []; 
    s_mpl_out_set = []; s_sl_out_set = [];
    hxb2_out_set = []; tf_out_set = []; cons_out_set = []
    mut_naive_out_set = []; # compare with TF
    reversion_out_set = []
    #mut_out_set = []; # consider the previous temporal state
    date_first_detected = [];
    freq_set = [[] for _ in 1:n_time_unique]
    for n in 1:length(idx_sort)
        i_raw = idx_sort[n]
        s_mpl = s_normed_MPL[i_raw]
        s_sl = s_normed_MPL[i_raw]
        i_poly = Int(ceil(i_raw/ q))

        a_poly_num = i_raw - (i_poly-1)*q
        a_poly = AA_set[a_poly_num]

        idx_csv = (csv_poly.polymorphic .== string(i_poly))
        i_poly_out = csv_poly.polymorphic[idx_csv][1]
        hxb2_out = csv_poly.HXB2[idx_csv][1]

        if( !( abs(s_mpl)<1e-7) )

            tf_out = csv_poly.TF[idx_csv][1]
            consensus_out = csv_poly.consensus[idx_csv][1]
            date_detected = get_first_detected(time_unique, x1, i_poly, a_poly_num)
            if((a_poly==consensus_out) && (a_poly!=tf_out)) push!(reversion_out_set, true) else push!(reversion_out_set, false) end
            #@show i_poly, a_poly, i_poly_out, hxb2_out, tf_out, consensus_out, date_detected
            push!(hxb2_out_set, hxb2_out)
            push!(i_poly_out_set, i_poly)
            push!(aa_out_set, a_poly)
            push!(tf_out_set, tf_out)
            push!(cons_out_set, consensus_out)
            push!(mut_naive_out_set, @sprintf("%s%s%s", tf_out, hxb2_out, a_poly))
            push!(date_first_detected, date_detected)
            push!(s_mpl_out_set, @sprintf("%.7f", s_mpl))
            push!(s_sl_out_set, @sprintf("%.7f", s_sl))
        
            for i_t in 1:n_time_unique
                push!(freq_set[i_t], @sprintf("%.3f", x1[i_t, i_poly, a_poly_num]))
            end
        
        end
    end
    (glycan_plus_set, glycan_minus_set) = get_glycan_plus_minus_AA_seq(seq_TF_aa, haxb2_TF, hxb2_out_set, aa_out_set)
    @printf("N+:%d, N-:%d, N±:%d\n", count(glycan_plus_set), count(glycan_minus_set), count(glycan_plus_set .* glycan_minus_set))
    (V1_set_temp, V2_set_temp, V3_set_temp, V4_set_temp, V5_set_temp, LD_set_temp, CD4BS_set_temp) = get_variable_site_true_false(hxb2_out_set)
    df = DataFrame(
        HXB2=hxb2_out_set, 
        polymorphic=i_poly_out_set, 
        PRO=aa_out_set, 
        TF=tf_out_set,
        consensus=cons_out_set,
        reversion=reversion_out_set,
        mutation=mut_naive_out_set, 
        date=date_first_detected,
        V1=V1_set_temp,
        V2=V2_set_temp,
        V3=V3_set_temp,
        V4=V4_set_temp,
        V5=V5_set_temp,
        LD=LD_set_temp,
        CD4BS=CD4BS_set_temp,
        glycan_plus=glycan_plus_set, 
        glycan_minus=glycan_minus_set, 
        s_MPL=s_mpl_out_set, 
        s_SL=s_sl_out_set, 
    )

    for i_t in 1:n_time_unique 
        df[!, @sprintf("f_at_%s", time_unique[i_t])] = freq_set[i_t]
    end
    CSV.write(@sprintf("../mpl/aa_csv/CH848/%s-poly.csv", fname_Human_RM[i_HRM]), df);
end

N+:24, N-:227, N±:5
N+:7, N-:54, N±:3
N+:12, N-:79, N±:3
N+:11, N-:90, N±:3
N+:8, N-:39, N±:2
N+:8, N-:61, N±:3
N+:5, N-:36, N±:1


# Marginalizing covariance and numerator; 

## CH505 

In [105]:
fname_Human_RM = ["703010505", "RM5695", "RM6072", "RM6701", "RM6699", "RM6697", "RM6070", "RM6703"];


In [106]:
i_HRM = 2
csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH505/index-%s.csv", fname_Human_RM[i_HRM])));
cov_raw = readdlm(@sprintf("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH505/covariance-%s-poly-AA.dat", fname_Human_RM[i_HRM]))
num_raw = readdlm(@sprintf("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH505/numerator-%s-poly-AA.dat", fname_Human_RM[i_HRM]));
coefficients_MPL = readdlm("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH505/"*fname_Human_RM[i_HRM]*"-poly-AA-MPL.dat");

In [107]:
# This takes 283.425596 sec.
#γ = 10 ;
#@time s_temp = (cov_raw + γ*I) \ num_raw;

In [108]:
# --- Define HXB2 set that are appeared on RMs at least once --- #
HXB2_set = []
for i_HRM in 2:length(fname_Human_RM)
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH505/index-%s.csv", fname_Human_RM[i_HRM])));
    for x in csv_poly.HXB2
        if(x ∉ HXB2_set )
            push!(HXB2_set, x)
        end
    end
end;

HXB2_set = copy(unique(HXB2_set));

# --- Set global "polymorphic" indicies across multiple RMs --- #
indicator_poly_set = [false for _ in HXB2_set];
TF_set = ["-" for _ in HXB2_set]
# A function that maps polymorphic index in RM to another "polymorphic of RMs"
for i_HRM in 2:length(fname_Human_RM)
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH505/index-%s.csv", fname_Human_RM[i_HRM])));
    non_NA_sites = csv_poly.HXB2[csv_poly.polymorphic .!= "NA"]
    for i in 1:length(HXB2_set)
        hxb2 = HXB2_set[i]
        if(hxb2 ∈ non_NA_sites)
            indicator_poly_set[i] = true
        end
    end    
  
end
poly_set = ["NA" for _ in HXB2_set];
L_RM = count(indicator_poly_set)
poly_set[indicator_poly_set] = string.(collect(1:L_RM));

# ------ make mapping between individual poly indicies to marginalized poly indicies ----- #;
HXB2_poly = HXB2_set[indicator_poly_set];
poly2poly_map = [[] for _ in 1:length(fname_Human_RM)]
num_set_temp = collect(1:length(HXB2_poly))
for i_HRM in 2:length(fname_Human_RM)
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH505/index-%s.csv", fname_Human_RM[i_HRM])));
    # --- Get indecies set that maps inidividual polymorphic sites to global polymorphic sites. --- #
    for x in csv_poly.HXB2[csv_poly.polymorphic .!= "NA"];
        idx = num_set_temp[HXB2_poly .== x]
        push!(poly2poly_map[i_HRM], idx[1])
    end;
end
# ----- Just for obtaining TF and aonsenssus----- #
TF_RMs_set = ["NA" for _ in HXB2_set]
consensus_RMs_set = ["NA" for _ in HXB2_set]
num_temp = collect(1:length(HXB2_set))
for i_HRM in 2:length(fname_Human_RM)
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH505/index-%s.csv", fname_Human_RM[i_HRM])));
    
    for k in 1:length(csv_poly.HXB2)
        if(csv_poly.polymorphic[k] != "NA")
            hxb2 = csv_poly.HXB2[k]
            idx = HXB2_set .== hxb2
            TF_RMs_set[num_temp[idx][1]] = string(csv_poly.TF[k][1])
            consensus_RMs_set[num_temp[idx][1]] = string(csv_poly.consensus[k][1])
        end 
    end
end


In [109]:
# ---- Marginalizing cov and num ---- #
cov_RM = zeros(q*L_RM, q*L_RM)
num_RM = zeros(q*L_RM);

for i_HRM in 2:length(fname_Human_RM)
    cov_raw = readdlm(@sprintf("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH505/covariance-%s-poly-AA.dat", fname_Human_RM[i_HRM]))
    num_raw = readdlm(@sprintf("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH505/numerator-%s-poly-AA.dat", fname_Human_RM[i_HRM]));
    poly_idx = copy(poly2poly_map[i_HRM]);
    @assert length(num_raw) == q*length(poly_idx)
    for i in 1:length(poly_idx)
        i_poly = poly_idx[i]
        num_RM[km.(i_poly, 1:q, q)] += num_raw[km.(i, 1:q, q)]
        for j in 1:length(poly_idx)
            j_poly = poly_idx[j]
            cov_RM[km.(i_poly, 1:q, q), km.(j_poly, 1:q, q)] += cov_raw[km.(i, 1:q, q), km.(j, 1:q, q)]
        end
    end
end

# ----- Write out integrated cov and num to CSV files ----- #
fout_num = open("../mpl/aa_out/CH505/numerator-RMs-poly-AA.dat", "w")
for x in num_RM
    println(fout_num, x)
end
close(fout_num)

fout_cov = open("../mpl/aa_out/CH505/covariance-RMs-poly-AA.dat", "w")
for i in 1:size(cov_RM,1)
    println(fout_cov, @sprintf("%s", join(cov_RM[i, :], " ")))
end
close(fout_cov);

df = DataFrame(
        HXB2=HXB2_set, 
        polymorphic=poly_set, 
        TF=TF_RMs_set, 
        consensus=consensus_RMs_set)
CSV.write("../mpl/aa_csv/CH505/index-RMs-poly.csv", df);

In [110]:
γ = 10
@time s_MPL_RM = (cov_RM + γ*I) \ num_RM;
@time s_SL_RM =  num_RM ./ (cov_RM[diagind(cov_RM)] .+ γ);

fout_mpl = open("../mpl/aa_out/CH505/RMs-poly-AA-MPL.dat", "w")
fout_sl = open("../mpl/aa_out/CH505/RMs-poly-AA-SL.dat", "w")
for x in s_MPL_RM
    println(fout_mpl, x)
end
close(fout_mpl)
#
for x in s_SL_RM
    println(fout_sl, x)
end
close(fout_sl);

 25.037920 seconds (9 allocations: 1.506 GiB, 0.03% gc time)
  0.005806 seconds (9 allocations: 157.375 KiB)


## CH848 

In [111]:
fname_Human_RM = ["703010505", "RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720" ];

In [112]:
# --- Define HXB2 set that are appeared on RMs at least once --- #
HXB2_set = []
for i_HRM in 2:length(fname_Human_RM)
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH848/index-%s.csv", fname_Human_RM[i_HRM])));
    for x in csv_poly.HXB2
        if(x ∉ HXB2_set )
            push!(HXB2_set, x)
        end
    end
end;

HXB2_set = copy(unique(HXB2_set));

# --- Set global "polymorphic" indicies across multiple RMs --- #
indicator_poly_set = [false for _ in HXB2_set];
TF_set = ["-" for _ in HXB2_set]
# A function that maps polymorphic index in RM to another "polymorphic of RMs"
for i_HRM in 2:length(fname_Human_RM)
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH848/index-%s.csv", fname_Human_RM[i_HRM])));
    non_NA_sites = csv_poly.HXB2[csv_poly.polymorphic .!= "NA"]
    for i in 1:length(HXB2_set)
        hxb2 = HXB2_set[i]
        if(hxb2 ∈ non_NA_sites)
            indicator_poly_set[i] = true
        end
    end    
  
end
poly_set = ["NA" for _ in HXB2_set];
L_RM = count(indicator_poly_set)
poly_set[indicator_poly_set] = string.(collect(1:L_RM));

# ------ make mapping between individual poly indicies to marginalized poly indicies ----- #;
HXB2_poly = HXB2_set[indicator_poly_set];
poly2poly_map = [[] for _ in 1:length(fname_Human_RM)]
num_set_temp = collect(1:length(HXB2_poly))
for i_HRM in 2:length(fname_Human_RM)
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH848/index-%s.csv", fname_Human_RM[i_HRM])));
    # --- Get indecies set that maps inidividual polymorphic sites to global polymorphic sites. --- #
    for x in csv_poly.HXB2[csv_poly.polymorphic .!= "NA"];
        idx = num_set_temp[HXB2_poly .== x]
        push!(poly2poly_map[i_HRM], idx[1])
    end;
end
# ----- Just for obtaining TF and aonsenssus----- #
TF_RMs_set = ["NA" for _ in HXB2_set]
consensus_RMs_set = ["NA" for _ in HXB2_set]
num_temp = collect(1:length(HXB2_set))
for i_HRM in 2:length(fname_Human_RM)
    csv_poly = DataFrame(CSV.File(@sprintf("../mpl/aa_csv/CH848/index-%s.csv", fname_Human_RM[i_HRM])));
    
    for k in 1:length(csv_poly.HXB2)
        if(csv_poly.polymorphic[k] != "NA")
            hxb2 = csv_poly.HXB2[k]
            idx = HXB2_set .== hxb2
            TF_RMs_set[num_temp[idx][1]] = string(csv_poly.TF[k][1])
            consensus_RMs_set[num_temp[idx][1]] = string(csv_poly.consensus[k][1])
        end 
    end
end

In [113]:
# ---- Marginalizing cov and num ---- #
cov_RM = zeros(q*L_RM, q*L_RM)
num_RM = zeros(q*L_RM);

for i_HRM in 2:length(fname_Human_RM)
    cov_raw = readdlm(@sprintf("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH848/covariance-%s-poly-AA.dat", fname_Human_RM[i_HRM]))
    num_raw = readdlm(@sprintf("/net/dali/home/barton/kais/MPL_AA_rhesus/HIV/CH848/numerator-%s-poly-AA.dat", fname_Human_RM[i_HRM]));
    poly_idx = copy(poly2poly_map[i_HRM]);
    @assert length(num_raw) == q*length(poly_idx)
    for i in 1:length(poly_idx)
        i_poly = poly_idx[i]
        num_RM[km.(i_poly, 1:q, q)] += num_raw[km.(i, 1:q, q)]
        for j in 1:length(poly_idx)
            j_poly = poly_idx[j]
            cov_RM[km.(i_poly, 1:q, q), km.(j_poly, 1:q, q)] += cov_raw[km.(i, 1:q, q), km.(j, 1:q, q)]
        end
    end
end

# ----- Write out integrated cov and num to CSV files ----- #
fout_num = open("../mpl/aa_out/CH848/numerator-RMs-poly-AA.dat", "w")
for x in num_RM
    println(fout_num, x)
end
close(fout_num)

fout_cov = open("../mpl/aa_out/CH848/covariance-RMs-poly-AA.dat", "w")
for i in 1:size(cov_RM,1)
    println(fout_cov, @sprintf("%s", join(cov_RM[i, :], " ")))
end
close(fout_cov);

# --- I need to include V1-V5, CD4, LD, Glycan alter.
# --> To compute the glycan I need to fix the TF sequence then identify the 
# consider to use the function. 
df = DataFrame(
        HXB2=HXB2_set, 
        polymorphic=poly_set, 
        TF=TF_RMs_set, 
        consensus=consensus_RMs_set
    )

CSV.write("../mpl/aa_csv/CH848/index-RMs-poly.csv", df);


In [114]:
γ = 10
@time s_MPL_RM = (cov_RM + γ*I) \ num_RM;
@time s_SL_RM =  num_RM ./ (cov_RM[diagind(cov_RM)] .+ γ);

fout_mpl = open("../mpl/aa_out/CH848/RMs-poly-AA-MPL.dat", "w")
fout_sl = open("../mpl/aa_out/CH848/RMs-poly-AA-SL.dat", "w")
for x in s_MPL_RM
    println(fout_mpl, x)
end
close(fout_mpl)
#
for x in s_SL_RM
    println(fout_sl, x)
end
close(fout_sl);

 25.524340 seconds (9 allocations: 1.669 GiB, 0.26% gc time)
  0.000484 seconds (9 allocations: 165.625 KiB)


# Making CSV for marginalized RMs.

In [115]:
fname_Human_RM = ["703010505", "RM5695", "RM6072", "RM6701", "RM6699", "RM6697", "RM6070", "RM6703"];

In [116]:
csv_poly = DataFrame(CSV.File("../mpl/aa_csv/CH505/index-RMs-poly.csv"))
# Read CSV file 
coefficients_MPL = readdlm("../mpl/aa_out/CH505/RMs-poly-AA-MPL.dat")
coefficients_SL = readdlm("../mpl/aa_out/CH505/RMs-poly-AA-SL.dat");


In [117]:
# Formatting sequencies and get frequency 
seq_TF = copy(csv_poly.TF[csv_poly.polymorphic .!= "NA"]);
seq_TF_num = [AA2NUM[x] for x in seq_TF];
L_poly = length(seq_TF_num)
# --- Normalization --- #
(s_normed_MPL, s_normed_SL) = normalize_coefficient_AA(coefficients_MPL, coefficients_SL, q, L_poly, seq_TF, seq_TF_num)
# --- Formatting CSV file --- #
idx_sort = sortperm(vec(s_normed_MPL), rev=true)

i_poly_out_set = []; aa_out_set = []; 
s_mpl_out_set = []; s_sl_out_set = [];
hxb2_out_set = []; tf_out_set = []; cons_out_set = []
mut_naive_out_set = []; # compare with TF

for n in 1:length(idx_sort)
    i_raw = idx_sort[n]
    s_mpl = s_normed_MPL[i_raw]
    s_sl = s_normed_MPL[i_raw]
    i_poly = Int(ceil(i_raw/ q))

    a_poly_num = i_raw - (i_poly-1)*q
    a_poly = AA_set[a_poly_num]

    idx_csv = (csv_poly.polymorphic .== string(i_poly))
    i_poly_out = csv_poly.polymorphic[idx_csv][1]
    hxb2_out = csv_poly.HXB2[idx_csv][1]

    if( !( abs(s_mpl)<1e-7) )
        tf_out = csv_poly.TF[idx_csv][1]
        consensus_out = csv_poly.consensus[idx_csv][1]
        push!(hxb2_out_set, hxb2_out)
        push!(i_poly_out_set, i_poly)
        push!(aa_out_set, a_poly)
        push!(tf_out_set, tf_out)
        push!(cons_out_set, consensus_out)
        push!(mut_naive_out_set, @sprintf("%s%s%s", tf_out, hxb2_out, a_poly))
        push!(s_mpl_out_set, @sprintf("%.7f", s_mpl))
        push!(s_sl_out_set, @sprintf("%.7f", s_sl))        
    end
end

df = DataFrame(
    HXB2=hxb2_out_set, 
    polymorphic=i_poly_out_set, 
    PRO=aa_out_set, 
    TF=tf_out_set,
    consensus=cons_out_set,
    mutation=mut_naive_out_set, 
    s_MPL=s_mpl_out_set, 
    s_SL=s_sl_out_set, 
)
CSV.write("../mpl/aa_csv/CH505/RMs-poly.csv", df);


## CH848

In [118]:
fname_Human_RM = ["703010505", "RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720" ];

In [119]:
csv_poly = DataFrame(CSV.File("../mpl/aa_csv/CH848/index-RMs-poly.csv"))
# Read CSV file 
coefficients_MPL = readdlm("../mpl/aa_out/CH848/RMs-poly-AA-MPL.dat")
coefficients_SL = readdlm("../mpl/aa_out/CH848/RMs-poly-AA-SL.dat");


In [120]:
# Formatting sequencies and get frequency 
seq_TF = copy(csv_poly.TF[csv_poly.polymorphic .!= "NA"]);
seq_TF_num = [AA2NUM[x] for x in seq_TF];
L_poly = length(seq_TF_num)
# --- Normalization --- #
(s_normed_MPL, s_normed_SL) = normalize_coefficient_AA(coefficients_MPL, coefficients_SL, q, L_poly, seq_TF, seq_TF_num)
# --- Formatting CSV file --- #
idx_sort = sortperm(vec(s_normed_MPL), rev=true)

i_poly_out_set = []; aa_out_set = []; 
s_mpl_out_set = []; s_sl_out_set = [];
hxb2_out_set = []; tf_out_set = []; cons_out_set = []
mut_naive_out_set = []; # compare with TF

for n in 1:length(idx_sort)
    i_raw = idx_sort[n]
    s_mpl = s_normed_MPL[i_raw]
    s_sl = s_normed_MPL[i_raw]
    i_poly = Int(ceil(i_raw/ q))

    a_poly_num = i_raw - (i_poly-1)*q
    a_poly = AA_set[a_poly_num]

    idx_csv = (csv_poly.polymorphic .== string(i_poly))
    i_poly_out = csv_poly.polymorphic[idx_csv][1]
    hxb2_out = csv_poly.HXB2[idx_csv][1]

    if( !( abs(s_mpl)<1e-7) )
        tf_out = csv_poly.TF[idx_csv][1]
        consensus_out = csv_poly.consensus[idx_csv][1]
        push!(hxb2_out_set, hxb2_out)
        push!(i_poly_out_set, i_poly)
        push!(aa_out_set, a_poly)
        push!(tf_out_set, tf_out)
        push!(cons_out_set, consensus_out)
        push!(mut_naive_out_set, @sprintf("%s%s%s", tf_out, hxb2_out, a_poly))
        push!(s_mpl_out_set, @sprintf("%.7f", s_mpl))
        push!(s_sl_out_set, @sprintf("%.7f", s_sl))        
    end
end

df = DataFrame(
    HXB2=hxb2_out_set, 
    polymorphic=i_poly_out_set, 
    PRO=aa_out_set, 
    TF=tf_out_set,
    consensus=cons_out_set,
    mutation=mut_naive_out_set, 
    s_MPL=s_mpl_out_set, 
    s_SL=s_sl_out_set, 
)
CSV.write("../mpl/aa_csv/CH848/RMs-poly.csv", df);


# Compute enrichment ratio for macaques and make a CSV file.

## SHIV.CH505

In [126]:
fname_Human_RM = ["703010505", "RM5695", "RM6072", "RM6701", "RM6699", "RM6697", "RM6070", "RM6703"];

In [127]:
list_set_HXB2 = []; list_set_s_MPL = []; list_set_date = []; list_set_mutation = []; list_set_reversion = []; list_set_RMs = []
list_set_N_plus = []; list_set_N_minus = []; list_set_N_shift = [];
list_set_V1 = []; list_set_V2 = []; list_set_V3 = []; list_set_V4 = []; list_set_V5 = []; list_set_LD = []; list_set_CD4BS = []
list_set_PRO = [] # This is protein but for the sake of consistency, I referr this nucleotide.
#i_HRM = 2
for i_HRM in 2:length(fname_Human_RM)
#for i_HRM in 2:2
    fname_in = @sprintf("../mpl/aa_csv/CH505/%s-poly.csv", fname_Human_RM[i_HRM])
    csv_poly = DataFrame(CSV.File( fname_in));
    @show fname_in
    append!(list_set_HXB2, copy(csv_poly.HXB2))
    append!(list_set_PRO, copy(csv_poly.PRO))
    append!(list_set_date, copy(csv_poly.date))
    append!(list_set_mutation, copy(csv_poly.mutation))
    append!(list_set_reversion, copy(csv_poly.reversion))
    append!(list_set_RMs, [fname_Human_RM[i_HRM] for _ in 1:length(csv_poly.HXB2)] )
    append!(list_set_s_MPL, copy(csv_poly.s_MPL))
    @printf("N+:%d, N-:%d, N±:%d\n", count(csv_poly.glycan_plus), count(csv_poly.glycan_minus), count(csv_poly.glycan_minus .* csv_poly.glycan_plus))
    glycan_plus_temp = zeros(length(csv_poly.glycan_plus)); glycan_plus_temp[csv_poly.glycan_plus] .= 1
    glycan_minus_temp = zeros(length(csv_poly.glycan_minus)); glycan_minus_temp[csv_poly.glycan_minus] .= 1
    glycan_shift_temp = zeros(length(csv_poly.glycan_plus)); glycan_shift_temp[csv_poly.glycan_plus .* csv_poly.glycan_minus] .= 1
    append!(list_set_N_plus, copy(glycan_plus_temp))
    append!(list_set_N_minus, copy(glycan_minus_temp))
    append!(list_set_N_shift, copy(glycan_shift_temp))
    append!(list_set_CD4BS, copy(csv_poly.CD4BS))
    append!(list_set_LD, copy(csv_poly.LD))
    append!(list_set_V1, copy(csv_poly.V1))
    append!(list_set_V2, copy(csv_poly.V2))
    append!(list_set_V3, copy(csv_poly.V3))
    append!(list_set_V4, copy(csv_poly.V4))
    append!(list_set_V5, copy(csv_poly.V5));
end


fname_in = "../mpl/aa_csv/CH505/RM5695-poly.csv"
N+:10, N-:47, N±:4
fname_in = "../mpl/aa_csv/CH505/RM6072-poly.csv"
N+:7, N-:63, N±:4
fname_in = "../mpl/aa_csv/CH505/RM6701-poly.csv"
N+:1, N-:15, N±:1
fname_in = "../mpl/aa_csv/CH505/RM6699-poly.csv"
N+:2, N-:16, N±:1
fname_in = "../mpl/aa_csv/CH505/RM6697-poly.csv"
N+:5, N-:43, N±:2
fname_in = "../mpl/aa_csv/CH505/RM6070-poly.csv"
N+:3, N-:38, N±:1
fname_in = "../mpl/aa_csv/CH505/RM6703-poly.csv"
N+:3, N-:21, N±:2


In [128]:
# Sort selection coefficients.
sort_indx = sortperm(list_set_s_MPL, rev=true);

df = DataFrame(
    HXB2_index = list_set_HXB2[sort_indx], 
    RM = list_set_RMs[sort_indx], 
    reversion = list_set_reversion[sort_indx], 
    mutation = list_set_mutation[sort_indx], 
    PRO = list_set_PRO[sort_indx],
    date = list_set_date[sort_indx], 
    s_MPL = list_set_s_MPL[sort_indx], 
    N_linked_glycan_plus_fr3 = list_set_N_plus[sort_indx], 
    N_linked_glycan_minus_fr3 = list_set_N_minus[sort_indx], 
    N_linked_glycan_shift_fr3 = list_set_N_shift[sort_indx], 
    LoopD = list_set_LD[sort_indx], 
    V1 = list_set_V1[sort_indx], 
    V2 = list_set_V2[sort_indx], 
    V3 = list_set_V3[sort_indx], 
    V4 = list_set_V4[sort_indx], 
    V5 = list_set_V5[sort_indx], 
    # The following is just copying other result to avoid error due to the absent of the process
    CD4BS = list_set_CD4BS[sort_indx], 
);

In [129]:
CSV.write("../out/summary-RMs-AA_CH505.csv", df); 

## SHIV.CH848

In [130]:
fname_Human_RM = ["703010505", "RM6163", "RM6167", "RM6700", "RM6713", "RM6714", "RM6720" ];

In [131]:
list_set_HXB2 = []; list_set_s_MPL = []; list_set_date = []; list_set_mutation = []; list_set_reversion = []; list_set_RMs = []
list_set_N_plus = []; list_set_N_minus = []; list_set_N_shift = [];
list_set_V1 = []; list_set_V2 = []; list_set_V3 = []; list_set_V4 = []; list_set_V5 = []; list_set_LD = []; list_set_CD4BS = []
list_set_PRO = [] # This is protein but for the sake of consistency, I referr this nucleotide.
#i_HRM = 2
for i_HRM in 2:length(fname_Human_RM)
    fname_in = @sprintf("../mpl/aa_csv/CH848/%s-poly.csv", fname_Human_RM[i_HRM])
    csv_poly = DataFrame(CSV.File( fname_in));
    append!(list_set_HXB2, copy(csv_poly.HXB2))
    append!(list_set_PRO, copy(csv_poly.PRO))
    append!(list_set_date, copy(csv_poly.date))
    append!(list_set_mutation, copy(csv_poly.mutation))
    append!(list_set_reversion, copy(csv_poly.reversion))
    append!(list_set_RMs, [fname_Human_RM[i_HRM] for _ in 1:length(csv_poly.HXB2)] )
    append!(list_set_s_MPL, copy(csv_poly.s_MPL))
    glycan_plus_temp = zeros(length(csv_poly.glycan_plus)); glycan_plus_temp[csv_poly.glycan_plus] .= 1
    glycan_minus_temp = zeros(length(csv_poly.glycan_minus)); glycan_minus_temp[csv_poly.glycan_minus] .= 1
    glycan_shift_temp = zeros(length(csv_poly.glycan_plus)); glycan_shift_temp[csv_poly.glycan_plus .* csv_poly.glycan_minus] .= 1
    append!(list_set_N_plus, copy(glycan_plus_temp))
    append!(list_set_N_minus, copy(glycan_minus_temp))
    append!(list_set_N_shift, copy(glycan_shift_temp))
    append!(list_set_CD4BS, copy(csv_poly.CD4BS))
    append!(list_set_LD, copy(csv_poly.LD))
    append!(list_set_V1, copy(csv_poly.V1))
    append!(list_set_V2, copy(csv_poly.V2))
    append!(list_set_V3, copy(csv_poly.V3))
    append!(list_set_V4, copy(csv_poly.V4))
    append!(list_set_V5, copy(csv_poly.V5));
end


In [132]:
# Sort selection coefficients.
sort_indx = sortperm(list_set_s_MPL, rev=true);

df = DataFrame(
    HXB2_index = list_set_HXB2[sort_indx], 
    RM = list_set_RMs[sort_indx], 
    reversion = list_set_reversion[sort_indx], 
    mutation = list_set_mutation[sort_indx], 
    PRO = list_set_PRO[sort_indx],
    date = list_set_date[sort_indx], 
    s_MPL = list_set_s_MPL[sort_indx], 
    N_linked_glycan_plus_fr3 = list_set_N_plus[sort_indx], 
    N_linked_glycan_minus_fr3 = list_set_N_minus[sort_indx], 
    N_linked_glycan_shift_fr3 = list_set_N_shift[sort_indx], 
    LoopD = list_set_LD[sort_indx], 
    V1 = list_set_V1[sort_indx], 
    V2 = list_set_V2[sort_indx], 
    V3 = list_set_V3[sort_indx], 
    V4 = list_set_V4[sort_indx], 
    V5 = list_set_V5[sort_indx], 
    # The following is just copying other result to avoid error due to the absent of the process
    CD4BS = list_set_CD4BS[sort_indx], 
);

In [133]:
CSV.write("../out/summary-RMs-AA_CH848.csv", df); 

# Making enrichment values 

In [134]:
include("../src/process_HIV_mutation_for_CSV.jl") ;

### CH505 

In [135]:
#csv_index_and_TF= DataFrame(CSV.File("../data/mpl/703010505-3-index.csv"));
csv_index_and_TF= DataFrame(CSV.File("../mpl/aa_csv/CH505/index-703010505.csv"));

csv_raw_SHIV_CH505 = DataFrame(CSV.File("../out/summary-RMs-AA_CH505.csv"));

categories = ["Rev.", "V1", "V2", "V3", "V4", "V5", "LD", "CD4", "PNG-shield", "PNG-holse", "PNG-shift", "PNG"];

α_threshold_RMs_set = sort([ [0.01 * i for i in 1:9]; [0.1 * i for i in 1:10] ], rev=true)


(x_fold_summary, log_P, α_summary, n_fold_summary, types_summary) = get_enrichment_and_pvalues_AA(csv_raw_SHIV_CH505, csv_index_and_TF, α_threshold_RMs_set);


(N_null_temp_all, N_null_all, N_sel_all) = (16094, 16094, 1433)
(n_null_LD, N_null_LD_all) = (171, 16094)
(n_null_V1, N_null_V1_all) = (323, 16094)
(n_null_V2, N_null_V2_all) = (760, 16094)
(n_null_V3, N_null_V3_all) = (665, 16094)
(n_null_V4, N_null_V4_all) = (646, 16094)
(n_null_V5, N_null_V5_all) = (171, 16094)
N_null_N_glycan_all = 1433
n_null_N_add = 26
n_null_N_rem = 243
n_null_N_sht = 15
n_null_N_any = 254
n_null_rev = 122
N_null_rev_all = 16094


In [136]:
df = DataFrame(
    categories=types_summary,
    fraction=[@sprintf("%.3f",x) for x in α_summary],
    num_cutoff=Int.(n_fold_summary),
    enrichment=x_fold_summary,
    log10_P = log_P
)
CSV.write("../out/enrichment_SHIV-CH505_multiply_fraction_AA.csv", df);



### CH848

In [137]:
#csv_index_and_TF= DataFrame(CSV.File("../data/mpl/703010848-3-index.csv"));
csv_index_and_TF= DataFrame(CSV.File("../mpl/aa_csv/CH848/index-703010848.csv"));

csv_raw_SHIV_CH848 = DataFrame(CSV.File("../out/summary-RMs-AA_CH848.csv"));

categories = ["Rev.", "V1", "V2", "V3", "V4", "V5", "LD", "CD4", "PNG-shield", "PNG-holse", "PNG-shift", "PNG"];

α_threshold_RMs_set = sort([ [0.01 * i for i in 1:9]; [0.1 * i for i in 1:10] ], rev=true)


(x_fold_summary, log_P, α_summary, n_fold_summary, types_summary) = get_enrichment_and_pvalues_AA(csv_raw_SHIV_CH848, csv_index_and_TF, α_threshold_RMs_set);


(N_null_temp_all, N_null_all, N_sel_all) = (16474, 16474, 2000)
(n_null_LD, N_null_LD_all) = (171, 16474)
(n_null_V1, N_null_V1_all) = (627, 16474)
(n_null_V2, N_null_V2_all) = (798, 16474)
(n_null_V3, N_null_V3_all) = (665, 16474)
(n_null_V4, N_null_V4_all) = (532, 16474)
(n_null_V5, N_null_V5_all) = (209, 16474)
N_null_N_glycan_all = 2000
n_null_N_add = 46
n_null_N_rem = 359
n_null_N_sht = 15
n_null_N_any = 390
n_null_rev = 116
N_null_rev_all = 16474


In [138]:
df = DataFrame(
    categories=types_summary,
    fraction=[@sprintf("%.3f",x) for x in α_summary],
    num_cutoff=Int.(n_fold_summary),
    enrichment=x_fold_summary,
    log10_P = log_P
)
CSV.write("../out/enrichment_SHIV-CH848_multiply_fraction_AA.csv", df);



# Check glycan distribution on sequences: 

In [6]:
"""
enrichment_CH505 = DataFrame(CSV.File("../out/enrichment_CH505_multiply_fraction.csv"));
enrichment_SHIVCH505 = DataFrame(CSV.File("../out/enrichment_grouped_SHIV_CH505_multiply_fraction.csv"));

enrichment_CH848 = DataFrame(CSV.File("../out/enrichment_CH848_multiply_fraction.csv"));
enrichment_SHIVCH848 = DataFrame(CSV.File("../out/enrichment_grouped_SHIV_CH848_multiply_fraction.csv"));
"""

"enrichment_CH505 = DataFrame(CSV.File(\"../out/enrichment_CH505_multiply_fraction.csv\"));\nenrichment_SHIVCH505 = DataFrame(CSV.File(\"../out/enrichment_grouped_SHIV_CH505_multiply_fraction.csv\"));\n\nenrichment_CH848 = DataFrame(CSV.File(\"../out/enrichment_CH848_multiply_fraction.csv\"));\nenrichment_SHIVCH848 = DataFrame(CSV.File(\"../out/enrichment_grouped_SHIV_CH848_multiply_fraction.csv\"));\n"

In [5]:
#field_key_list = ["Reversion", "V1", "V2", "V3", "V4", "V5", "LD", "CD4BS"]
#field_key_list2 = ["PNG", "Shield", "Hole", "Shift"]
#field_key_list = ["Reversion", "V1", "V2", "V3", "V4", "V5", "LD", "CD4BS"]
#field_key_list2 = ["PNG", "Shield", "Hole", "Shift"]

#field_key_list = ["Reversion", "V1", "V2", "V3", "V4", "V5", "LD", "CD4BS"]
#field_key_list2 = ["PNG", "Shield", "Hole", "Shift"]
#field_key_list = ["Reversion", "V1", "V2", "V3", "V4", "V5", "LD", "CD4BS"]
#field_key_list2 = ["PNG", "Shield", "Hole", "Shift"]